In [1]:
import matplotlib.pyplot as plt
import random as r
import pprint as pp
import numpy as np
import csv

In [2]:
# Define type of slice and its parameters
slices_type = {
    'eMBB' : {
        'bandwidth_max': 100000000, #max bandwidth which BS can give to this slice
        'client_weight': 0.3, # Portion of clients which use this type of slice
        'band_guaranteed' : 0, # based on quality of service
        'min': 4000000, 
        'max': 500000000 # how much bps can require this type of slice
      },
      'URLLC' : {
        'bandwidth_max': 10000000,
        'client_weight': 0.2,
        'band_guaranteed' : 5000000,
        'min': 5000000,
        'max': 8000000
      },
      'MIoT' : {
        'bandwidth_max': 10000000,
        'client_weight': 0.15,
        'band_guaranteed' : 1000000,
        'min': 1000000, 
        'max': 8000000
      },
      'mMTC' : {
        'bandwidth_max': 10000000,
        'client_weight': 0.15,
        'band_guaranteed' : 1000000,
        'min': 1000000, 
        'max': 8000000
      },
      'voice' : {
        'bandwidth_max': 1000000,
        'client_weight': 0.2,
        'band_guaranteed' : 500000,
        'min': 4000000, 
        'max': 8000000
     }
}
          
# Define base station parameters
basestation = {
    'capacity': 20000000000,   #20Gbps
    'coverage': 250,
    'ratios': { # portion of capacity dedicated to various slice type
        'URLLC': 0.05,
        'eMBB': 0.55,
        'mMTC': 0.15,
        'voice': 0.1,
        'MIoT': 0.15
    },
    'x': 300,
    'y': 300
}

# Define mobility patterns and its parameters
mobility_patterns = {
  'car': {
    'distribution': 'normal',
    'params':(0,7),
    'client_weight': 0.10
  },
  'walk': {
    'distribution': 'randint',
    'params': (-1, 1),
    'client_weight': 0.40
  },
  'stationary': {
    'distribution': 'normal',
    'params': (0, 0.1),
    'client_weight': 0.20
  },
  'publictransport': {
    'distribution': 'randint',
    'params': (-4, 4),
    'client_weight': 0.10
  },
  'slackperson':{
    'distribution': 'randint',
    'params':(0, 1),
    'client_weight': 0.20
  }
}

In [3]:
# Define obj Client
class Client:
    def __init__(self, ID, x, y, mobility_pattern, freq, slice_type, bs, slice_cap = None):
        self.ID = ID
        self.x = x
        self.y = y
        self.mobility_pattern = mobility_pattern
        self.freq = freq
        self.slice_type = slice_type
        self.bs = bs
        self.usage_rem = 0
        self.last_usage = 0
        self.connected = False
        
    #def step_0(self):
    
    def step_1(self):
        if self.usage_rem > 0:
            if self.connected:
                self.start_consume()
            else:
                self.connect() # if connection refused, usage remaining in memory
        else:
            if self.connected:
                self.disconnect()
            else:
                self.generate_usage_and_connect()
        
    def step_2(self):
        if self.connected and self.last_usage > 0:
            self.release_consume()
            if self.usage_rem == 0:
                self.disconnect()
    
    def step_3(self):
        x, y = self.move()
        self.x += x
        self.y += y
        if self.bs is not None:
            if not inRange(self, bs):
                self.disconnect()       
                
    def generate_usage_and_connect(self):
        if self.slice_type is not None:
            # Generate new usage
            self.usage_rem = self.generate()
            self.connect()
            if self.slice_type == 'URLLC':
                print(f"Client_{self.ID}, coordinates : ({self.x}, {self.y}) requests {self.usage_rem} usage to BS{self.bs.ID} to slice {self.slice_type}")     

    def connect(self):
        s = self.slice_type
        
        if self.connected:
            return
        
        if self.is_avaliable():         # threshold for usage
            self.connected = True
            bs.connected_users += 1
            
            if s == 'eMBB':
                bs.embb_users += 1
            if s == 'URLLC':
                bs.urllc_users += 1
                print("AGGIUNTO USER", bs.urllc_users)
            if s == 'MIoT':
                bs.miot_users += 1
            if s == 'mMTC':
                bs.mmtc_users += 1
            if s == 'voice':
                bs.voice_users += 1
                
            if self.slice_type == 'URLLC':  
                print(f"Client {self.ID}, coordinates : ({self.x}, {self.y}) connected to slice {s} @ BS{self.bs.ID}")
        else:
            self.connected = False
            if self.slice_type == 'URLLC':
                print(f"Client {self.ID}, coordinates : ({self.x}, {self.y}) connection refused to slice {s} @ BS{self.bs.ID}")
        
    def disconnect(self):
        self.connected = False
        s = self.slice_type
            
        bs.connected_users -= 1

        if s == 'eMBB':
            bs.embb_users -= 1
        if s == 'URLLC':
            bs.urllc_users -= 1
            print("DELETE USER", bs.urllc_users)
        if s == 'MIoT':
            bs.miot_users -= 1
        if s == 'mMTC':
            bs.mmtc_users -= 1
        if s == 'voice':
            bs.voice_users -= 1
                
        self.connected = False
        
        if self.slice_type == 'URLLC':   
            print(f"Client {self.ID}, coordinates: ({self.x},{self.y}) disconnected from slice {self.slice_type} @ BS{self.bs.ID} ")
    
    def start_consume(self):
        amount = min(self.usage_rem, self.get_consumable_share())    # bandwidth allocation
        self.last_usage = amount
        if self.slice_type == 'URLLC':
            print(f"Client {self.ID}, coordinates: ({self.x}, {self.y}) gets {amount} usage from @ BS{self.bs.ID} for slice {self.slice_type}")
           
    def release_consume(self):
        # release resources
        if self.last_usage > 0:
            self.usage_rem -= self.last_usage
            if self.slice_type == 'URLLC':
                print(f"Client {self.ID}, coordinates: ({self.x}, {self.y}), releases {self.last_usage} usage to BS{self.bs.ID}")
            self.last_usage = 0
        
    # update client coordinates    
    def move(self):   
        distr = self.mobility_pattern.distribution
        params = self.mobility_pattern.params
        if distr == "randint":
            delta_x = r.randint(params[0], params[1])
            delta_y = r.randint(params[0], params[1])
            return delta_x, delta_y
        if distr == "normal":
            delta_x = np.random.normal(params[0], params[1])
            delta_y = np.random.normal(params[0], params[1])
            return delta_x, delta_y 
        
    # generate client consume   
    def generate(self):
        s = self.slice_type
        cicciolina = r.randint(slices_type[s]['min'], slices_type[s]['max'])
        return cicciolina
    
    def is_avaliable(self):
        global capacity
        s = self.slice_type
        
        if s == 'eMBB':
            bandwidth_next = capacity[s] / (bs.embb_users + 1)
        if s == 'URLLC':
            bandwidth_next = capacity[s] / (bs.urllc_users + 1)
        if s == 'MIoT':
            bandwidth_next = capacity[s] / (bs.miot_users + 1)
        if s == 'mMTC':
            bandwidth_next = capacity[s] / (bs.mmtc_users + 1)
        if s == 'voice':
            bandwidth_next = capacity[s] / (bs.voice_users + 1)
       
        if bandwidth_next < slices_type[s]['band_guaranteed']:
            return False

        return True
      
    def get_consumable_share(self):
        global capacity
        s = self.slice_type

        if s == 'eMBB':
            if bs.embb_users <= 0:
                return int(min(capacity[s], slices_type[s]['bandwidth_max']))
            else: 
                return int(min(capacity[s]/bs.embb_users, slices_type[s]['bandwidth_max']))
        
        if s == 'URLLC':
            if bs.urllc_users <= 0:
                return int(min(capacity[s], slices_type[s]['bandwidth_max']))
            else:
                return int(min(capacity[s]/bs.urllc_users, slices_type[s]['bandwidth_max']))
        
        if s == 'MIoT':
            if bs.miot_users <= 0:
                return int(min(capacity[s], slices_type[s]['bandwidth_max']))
            else:
                return int(min(capacity[s]/bs.miot_users, slices_type[s]['bandwidth_max']))
        
        if s == 'mMTC':
            if bs.mmtc_users <= 0:
                return int(min(capacity[s], slices_type[s]['bandwidth_max']))
            else: 
                return int(min(capacity[s]/bs.mmtc_users, slices_type[s]['bandwidth_max']))
        
        if s == 'voice':
            if bs.voice_users <= 0:
                return int(min(capacity[s], slices_type[s]['bandwidth_max']))
            else:
                return int(min(capacity[s]/bs.voice_users, slices_type[s]['bandwidth_max']))
        

In [4]:
# Define obj Mobility 
class Mobility:
    def __init__(self, name, distribution, params):
        self.name = name
        self.distribution = distribution
        self.params = params
        

In [5]:
# Define obj BaseStation
class BaseStation:
    def __init__(self,ID, x, y, coverage, capacity, ratios):
        self.ID =ID
        self.x = x
        self.y = y
        self.coverage = coverage
        self.capacity = capacity
        self.ratios = ratios
        self.pot_band_remaining = capacity
        self.band_remaining = capacity
        self.connected_users = 0
        self.embb_users = 0
        self.urllc_users = 0
        self.miot_users = 0
        self.mmtc_users = 0
        self.voice_users = 0
        

In [6]:
def get_random_mobility_pattern(vals, mobility_patterns):
    i = 0
    j = r.random()

    while vals[i] < j:
        i += 1

    return mobility_patterns[i]

def get_random_slice_type(vals):
    i = 0
    j = r.random()
    s = ''

    while vals[i] < j:
        i += 1
    
    if i == 0:
       s = 'eMBB'
    if i == 1:
       s = 'URLLC' 
    if i == 2:
       s = 'MIoT' 
    if i == 3:
       s = 'mMTC' 
    if i == 4:
       s = 'voice' 
    
    return s

def inRange(client, bs):
    x_client = client.x
    y_client = client.y
    x_bs = bs.x
    y_bs = bs.y
    if (abs(x_client - x_bs) < bs.coverage) and (abs(y_client - y_bs) < bs.coverage):
        return True
        print('true')
    else:
        return False
        print('false')

collected, slice_weights = 0, []
for key, value in slices_type.items():
    collected += value['client_weight']
    slice_weights.append(collected)

collected, mb_weights = 0, []
for key, value in mobility_patterns.items():
    collected += value['client_weight']
    mb_weights.append(collected)
    
m_patterns = []
for name, mb in mobility_patterns.items():
    mobility_pattern = Mobility(name, mb['distribution'], mb['params'])
    m_patterns.append(mobility_pattern)   


In [7]:
# Define class Request
class Request:
    def __init__(self, id_client, type_slice, traffic_request, traffic_left):
        self.id_client = id_client
        self.type_slice = type_slice
        self.traffic_request = traffic_request
        self.traffic_left = traffic_left
        
#Define class csv
class CsvData:
    def __init__(self, step, r_embb, g_embb, connect_embb, r_urllc, g_urllc, connect_urllc, r_miot, g_miot, connect_miot, r_mmtc, g_mmtc, connect_mmtc, r_voice, g_voice, connect_voice):
        self.step = step
        self.r_embb = r_embb
        self.g_embb = g_embb
        self.connect_embb = connect_embb
        self.r_urllc = r_urllc
        self.g_urllc = g_urllc
        self.connect_urllc = connect_urllc
        self.r_miot = r_miot
        self.g_miot = g_miot
        self.connect_miot = connect_miot
        self.r_mmtc = r_mmtc
        self.g_mmtc = g_mmtc
        self.connect_mmtc = connect_mmtc
        self.r_voice = r_voice
        self.g_voice = g_voice
        self.connect_voice = connect_voice
        

In [8]:
# Round 0: create network area, place base station and clients
num_clients = 2000
simulation_time = 3
x_area = 600
y_area = 600

bs = BaseStation(0, basestation.get('x'), basestation.get('y'), basestation.get('coverage'), basestation.get('capacity'), basestation.get('ratios'))

clients = []
for i in range(num_clients):
    x = r.randint(0, x_area)
    y = r.randint(0, y_area)
    freq = r.random()
    
    mobility_pattern = get_random_mobility_pattern(mb_weights, m_patterns)
    slice_type = get_random_slice_type(slice_weights) 
    
    c = Client(i, x, y, mobility_pattern, freq, slice_type, bs)
    clients.append(c)


In [9]:
active_clients = []
request_list = [None] * len(clients) # create fixed size list to manage the request
csv_final = []
clients_inRange = []
for i in range(num_clients):
    if inRange(clients[i], bs):
        clients_inRange.append(clients[i])

for n in range(simulation_time):
    
    # manage number active clients
    active = []
    new_connection = r.randint(0, len(clients_inRange))
    active = r.sample(clients_inRange, new_connection)
    
    for item in active:
        if item.slice_type == 'URLLC':
            print(item.ID, item.connected)
    
    for item in active:
        if item.connected == False:
            active_clients.append(item)
    
    capacity = { 'eMBB' : 11000000000,
                 'URLLC' :1000000000,
                 'MIoT' : 3000000000,
                 'mMTC' : 3000000000,
                 'voice' : 2000000000
               }
    
    print('STEP', n)
    
    for item in active_clients:   
        # step 1 and create request
        item.step_1()
        
        req = Request(item.ID, item.slice_type,item.usage_rem, item.usage_rem)
        request_list[item.ID] = req
  
    for item in active_clients:
        # step 2 and update request (delete non active user, and update traffic)
        item.step_2()
        request_list[item.ID].traffic_left = item.usage_rem
    
    user_embb, user_urllc, user_miot, user_mmtc, user_voice = 0, 0, 0, 0, 0
    req_embb, req_urllc, req_miot, req_mmtc, req_voice = 0, 0, 0, 0, 0
    giv_embb, giv_urllc, giv_miot, giv_mmtc, giv_voice = 0, 0, 0, 0, 0
    for item in request_list:
     if item is not None:
        if item.type_slice == 'eMBB':
            if item.traffic_request - item.traffic_left != 0:
                req_embb += item.traffic_request
                giv_embb += item.traffic_request - item.traffic_left 
                user_embb += 1
            else:
                req_embb += 0
                giv_embb += 0
                
        if item.type_slice == 'URLLC' :
            if item.traffic_request - item.traffic_left != 0:
                req_urllc += item.traffic_request
                giv_urllc += item.traffic_request - item.traffic_left
                user_urllc += 1
            else:
                req_urllc += 0
                giv_urllc += 0
 
        if item.type_slice == 'MIoT':
            if item.traffic_request - item.traffic_left != 0:
                req_miot += item.traffic_request
                giv_miot += item.traffic_request - item.traffic_left
                user_miot += 1
            else:
                req_miot += 0
                giv_miot += 0
 
        if item.type_slice == 'mMTC':
            if item.traffic_request - item.traffic_left != 0:
                req_mmtc += item.traffic_request
                giv_mmtc += item.traffic_request - item.traffic_left
                user_mmtc += 1
            else:
                req_mmtc += 0
                giv_mmtc += 0

        if item.type_slice == 'voice':
            if item.traffic_request - item.traffic_left != 0:
                req_voice += item.traffic_request
                giv_voice += item.traffic_request - item.traffic_left
                user_voice += 1
            else:
                req_voice += 0
                giv_voice += 0
    
    csv_data = CsvData(n, req_embb, giv_embb, user_embb, req_urllc, giv_urllc, user_urllc, req_miot, giv_miot, user_miot, req_mmtc, giv_mmtc, user_mmtc, req_voice, giv_voice, user_voice)  
    csv_final.append(csv_data)

    for item in active_clients:
        # step 3 and delete also user not in range
        item.step_3()
    
    for item in active_clients:
        if item.connected == False:
            request_list[item.ID] = None
            active_clients.remove(item)
    
    ''' for item in active_clients:
        if item.slice_type == 'URLLC':
            print(item.ID)'''
            

1426 False
1556 False
650 False
968 False
225 False
1311 False
243 False
1754 False
1959 False
509 False
582 False
194 False
498 False
1845 False
362 False
966 False
59 False
671 False
4 False
1025 False
1961 False
11 False
412 False
1455 False
1795 False
291 False
125 False
954 False
248 False
570 False
1603 False
13 False
114 False
1721 False
1104 False
819 False
576 False
401 False
1274 False
1068 False
407 False
405 False
556 False
503 False
1701 False
337 False
1859 False
1162 False
197 False
1920 False
506 False
516 False
1060 False
706 False
1312 False
1452 False
841 False
1724 False
1935 False
107 False
733 False
847 False
1377 False
1088 False
1194 False
162 False
1816 False
870 False
1053 False
1050 False
1883 False
1723 False
604 False
1502 False
193 False
428 False
185 False
1165 False
1255 False
1226 False
330 False
1591 False
1746 False
1285 False
245 False
840 False
1180 False
639 False
911 False
1193 False
1544 False
371 False
960 False
1072 False
524 False
1152 False
9

1852 False
1808 False
1009 False
1088 True
847 True
966 True
94 False
1284 False
639 True
1374 False
1260 False
1469 False
401 True
524 True
213 False
1394 False
1845 True
1226 True
1426 True
1844 False
1401 False
1513 False
674 False
1235 False
1724 True
1131 False
1682 False
312 True
1746 True
345 False
1215 True
604 True
418 False
703 False
1723 True
650 True
1287 False
1767 False
189 False
100 True
408 True
428 True
23 False
706 True
363 True
1025 True
1701 True
353 False
1053 True
1413 False
1180 True
1050 True
1556 True
1896 False
1416 False
1544 True
1388 False
490 False
1502 True
1360 False
1162 True
13 True
390 False
1312 True
761 False
1285 True
1979 False
1954 False
498 True
1253 True
139 False
632 False
1443 False
1065 False
1959 True
475 False
1350 False
1384 False
1294 False
558 False
1603 True
1322 True
1958 False
337 True
257 False
1769 False
12 True
1194 True
876 False
840 True
891 False
1714 False
1165 True
359 False
1663 True
197 True
841 True
1377 True
1022 True
327

AGGIUNTO USER 159
Client 1420, coordinates : (331, 374) connected to slice URLLC @ BS0
Client_1420, coordinates : (331, 374) requests 6558114 usage to BS0 to slice URLLC
AGGIUNTO USER 160
Client 460, coordinates : (478, 91) connected to slice URLLC @ BS0
Client_460, coordinates : (478, 91) requests 7065622 usage to BS0 to slice URLLC
AGGIUNTO USER 161
Client 1695, coordinates : (443, 165) connected to slice URLLC @ BS0
Client_1695, coordinates : (443, 165) requests 5669366 usage to BS0 to slice URLLC
AGGIUNTO USER 162
Client 1070, coordinates : (197, 193) connected to slice URLLC @ BS0
Client_1070, coordinates : (197, 193) requests 6438805 usage to BS0 to slice URLLC
AGGIUNTO USER 163
Client 1946, coordinates : (290, 356) connected to slice URLLC @ BS0
Client_1946, coordinates : (290, 356) requests 6575847 usage to BS0 to slice URLLC
AGGIUNTO USER 164
Client 200, coordinates : (532, 542) connected to slice URLLC @ BS0
Client_200, coordinates : (532, 542) requests 6411483 usage to BS0 t

1935 False
327 False
107 False
196 True
959 False
1180 False
245 False
92 False
1645 False
1322 False
289 False
341 True
1350 False
582 False
968 False
615 False
941 True
1883 False
1723 False
4 False
114 False
1035 False
1372 True
933 False
1958 False
876 False
804 False
1603 False
1898 True
200 True
1260 True
330 False
460 True
225 False
1009 True
1587 True
1374 True
1695 True
1209 False
1974 False
0 False
1599 False
1845 False
452 True
1377 False
1025 False
1740 True
1827 False
1285 False
1410 False
1255 False
761 False
903 False
954 False
353 False
1123 True
1131 True
666 False
1819 True
1226 False
650 False
1492 True
1294 False
966 False
1312 False
1452 False
727 True
726 True
1144 False
1513 True
1065 False
STEP 2
AGGIUNTO USER 79
Client 1556, coordinates : (288.10573188039234, 552.9678966751344) connected to slice URLLC @ BS0
Client_1556, coordinates : (288.10573188039234, 552.9678966751344) requests 6788016 usage to BS0 to slice URLLC
AGGIUNTO USER 80
Client 650, coordinates : 

AGGIUNTO USER 147
Client 1544, coordinates : (210.0881628142542, 450.9167635449988) connected to slice URLLC @ BS0
Client_1544, coordinates : (210.0881628142542, 450.9167635449988) requests 5662721 usage to BS0 to slice URLLC
AGGIUNTO USER 148
Client 1515, coordinates : (60, 536) connected to slice URLLC @ BS0
Client_1515, coordinates : (60, 536) requests 7543541 usage to BS0 to slice URLLC
Client 162, coordinates: (222, 69) gets 6198886 usage from @ BS0 for slice URLLC
AGGIUNTO USER 149
Client 556, coordinates : (85, 166) connected to slice URLLC @ BS0
Client_556, coordinates : (85, 166) requests 5591414 usage to BS0 to slice URLLC
AGGIUNTO USER 150
Client 139, coordinates : (157, 289) connected to slice URLLC @ BS0
Client 1068, coordinates: (200, 250) gets 6132708 usage from @ BS0 for slice URLLC
AGGIUNTO USER 151
Client 1426, coordinates : (495.98494726749766, 230.90343114208312) connected to slice URLLC @ BS0
Client_1426, coordinates : (495.98494726749766, 230.90343114208312) reque

DELETE USER 152
Client 128, coordinates: (161,467) disconnected from slice URLLC @ BS0 
Client 753, coordinates: (164, 179), releases 6215922 usage to BS0
DELETE USER 151
Client 753, coordinates: (164,179) disconnected from slice URLLC @ BS0 
Client 1758, coordinates: (422, 184), releases 6329581 usage to BS0
DELETE USER 150
Client 1758, coordinates: (422,184) disconnected from slice URLLC @ BS0 
Client 1637, coordinates: (143, 182), releases 6438150 usage to BS0
DELETE USER 149
Client 1637, coordinates: (143,182) disconnected from slice URLLC @ BS0 
Client 1420, coordinates: (332, 373), releases 6558114 usage to BS0
DELETE USER 148
Client 1420, coordinates: (332,373) disconnected from slice URLLC @ BS0 
Client 460, coordinates: (479, 91), releases 7065622 usage to BS0
DELETE USER 147
Client 460, coordinates: (479,91) disconnected from slice URLLC @ BS0 
Client 1695, coordinates: (444, 164), releases 5669366 usage to BS0
DELETE USER 146
Client 1695, coordinates: (444,164) disconnected 

In [10]:
with open('input.csv', mode='w') as csv_file:
    fieldnames = ['embb_rate', 'embb_users', 'embb_req', 'urllc_rate', 'urllc_users', 'urllc_req', 'miot_rate', 'miot_users', 'miot_req', 'mmtc_rate', 'mmtc_users', 'mmtc_req', 'voice_rate', 'voice_users', 'voice_req']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    writer.writeheader()
    for item in csv_final:
        writer.writerow({'embb_rate': item.g_embb, 'embb_users': item.connect_embb, 'embb_req': item.r_embb, 'urllc_rate': item.g_urllc, 'urllc_users': item.connect_urllc, 'urllc_req': item.r_urllc, 'miot_rate': item.g_miot, 'miot_users': item.connect_miot, 'miot_req': item.r_miot, 'mmtc_rate': item.g_mmtc, 'mmtc_users': item.connect_mmtc, 'mmtc_req': item.r_mmtc, 'voice_rate': item.g_voice, 'voice_users': item.connect_voice, 'voice_req': item.r_voice})


In [11]:
584
695
1784
902
291
1431
913
1283
1586
1632
1919
1015
435
933
1738
680
692
332
771
1089
1580
797
1840
375
1605
1604
1417
388
1067
1096
996
119
634
108
1663
1948
234
1701
1841
484
1221
536
1055
697
1422
1078
1760
1424
1693
1832
1570
159
1918

1918